In [8]:
import pandas as pd
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split

## 数据预处理

In [10]:
corpus_data = pd.read_csv( "./data/corpus.tsv", sep="\t", names=["gid", "gname"])
print(f"商品标题corpus_data = {corpus_data.shape}")
train_data = pd.read_csv("./data/train.query.txt", sep="\t", names=["qid", "query"])
print(f"用户查询train_data = {train_data.shape}")
qrels = pd.read_csv("./data/qrels.train.tsv", sep="\t", names=["qid", "gid"])
print(f"查询与商品点击关系qrels = {qrels.shape}")
test_data = pd.read_csv("./data/dev.query.txt", sep="\t", names=["qid", "query"])
print(f"用户查询test_data = {test_data.shape}")

corpus_data = corpus_data.set_index("gid")
train_data = train_data.set_index("qid")
qrels = qrels.set_index("qid")
test_data = test_data.set_index("qid")

商品标题corpus_data = (1001500, 2)
用户查询train_data = (100000, 2)
查询与商品点击关系qrels = (100000, 2)
用户查询test_data = (1000, 2)


In [11]:
query2corpus = pd.DataFrame(data=corpus_data.loc[qrels.gid]['gname'].values,index=train_data.loc[qrels.index]['query'],columns=['gname'])

In [ ]:
X_train,X_val = train_test_split(query2corpus,test_size=0.1)

In [ ]:
X_train.to_csv("./data/X_train.csv")